In [1]:
import os
import sys
import importlib
from datetime import datetime

parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from dotenv import load_dotenv
_ = load_dotenv()

from langchain_core.messages import SystemMessage, HumanMessage
import agents.supervisor_agent
import agents.action_execution_agent

# RELOAD MODULES to ensure we use the latest code updates
importlib.reload(agents.action_execution_agent)
importlib.reload(agents.supervisor_agent)

from agents.supervisor_agent import ChatSupervisorAgent
from agents.action_execution_agent import ActionExecutionAgent

Load Azure Models

In [2]:
from agents.llm_model import AzureModelProvider
provider = AzureModelProvider()

llm = provider.get_primary_model()
fast_llm = provider.get_light_model()

In [3]:
llm.invoke([SystemMessage(content="Ready to assist?")]).content

'Hello! How can I assist you today?'

Import Prompts

In [4]:
# Load the intent classifier prompt from the text file
with open(os.path.join(parent_dir, "prompts", "intent_classifier_prompt.txt"), "r") as f:
    intent_classifier_prompt = f.read()

# Load the action execution prompt from the text file
with open(os.path.join(parent_dir, "prompts", "action_execution_prompt.txt"), "r") as f:
    action_execution_prompt = f.read()


## Agent Testing

Action Execution

In [5]:
message = "what's my schedule for next week?"
# message = "Can I see my latest payslip?"
# message = "When does my contract end?"
# message = "Show me my current employment agreement?"
# message = "How many vacation hours total do I have?"
# message = "Can you show me all monetary balances related to vacation?"

# message = "How can I call in sick?"
# message = "What's Trump's first name?"

In [6]:
action_agent = ActionExecutionAgent(llm, action_execution_prompt)

In [7]:
from datetime import datetime

test_state = {
    "date": datetime.now().strftime("%Y-%m-%d"),
    "messages": [HumanMessage(content=message)],
    "candidate_id": os.environ.get("KENTRO_TEST_CANDIDATE_ID"),
    "employee_number": os.environ.get("PLANBITION_TEST_EMPLOYEE_NUM"),
    "next_action": "ActionExecutionAgent",
    "retrieved_data": "",
    "error_message": "",
    "attempts": 0,
    "max_retries": 3
}

In [8]:
action_agent.run(test_state)

---ACTION EXECUTION AGENT: Starting run() [ID: 4551162048]---
DEBUG: Processing message: what's my schedule for next week?...
---Invoking LLM (with tools)---
---Executing Tool: get_schedule---
DEBUG: Tool Args: {'employee_number': 'RE0036443', 'start_date': '2025-11-24', 'end_date': '2025-11-30'}


Error calling ScheduleEmployeeShiftDemand: 500 Server Error: Internal Server Error for url: https://uat-rest.planbition.nl/api/ScheduleEmployeeShiftDemand?filter=contains%28EmployeeNumber%2C+%27RE0036443%27%29&PageNumber=1&PageSize=500
Error in get_employee_schedule: 500 Server Error: Internal Server Error for url: https://uat-rest.planbition.nl/api/ScheduleEmployeeShiftDemand?filter=contains%28EmployeeNumber%2C+%27RE0036443%27%29&PageNumber=1&PageSize=500


---Tool execution completed. Result length: 2---


{'retrieved_data': '[]', 'error_message': None}

ChatSupervisor

In [9]:
agent = ChatSupervisorAgent(model=llm, system_prompt=intent_classifier_prompt)

In [10]:
test_state = {
    "date": "",
    "messages": [HumanMessage(content=message)],
    "candidate_id": os.environ.get("KENTRO_TEST_CANDIDATE_ID"),
    "employee_number": os.environ.get("PLANBITION_TEST_EMPLOYEE_NUM"),
    "next_action": "",
    "retrieved_data": "",
    "error_message": "",
    "attempts": 0,
    "max_retries": 3
}

test_state

{'date': '',
 'messages': [HumanMessage(content="what's my schedule for next week?", additional_kwargs={}, response_metadata={})],
 'candidate_id': '100102195',
 'employee_number': 'RE0036443',
 'next_action': '',
 'retrieved_data': '',
 'error_message': '',
 'attempts': 0,
 'max_retries': 3}

In [11]:
# Run
result = agent.graph.invoke(test_state)

---SUPERVISOR: Classifying Intent---
Intent: ActionExecutionAgent
---ACTION EXECUTION AGENT: Starting run() [ID: 4550687184]---
DEBUG: Processing message: what's my schedule for next week?...
---Invoking LLM (with tools)---
---Executing Tool: get_schedule---
DEBUG: Tool Args: {'employee_number': 'RE0036443', 'start_date': '2024-06-10', 'end_date': '2024-06-16'}


Error calling ScheduleEmployeeShiftDemand: 500 Server Error: Internal Server Error for url: https://uat-rest.planbition.nl/api/ScheduleEmployeeShiftDemand?filter=contains%28EmployeeNumber%2C+%27RE0036443%27%29&PageNumber=1&PageSize=500
Error in get_employee_schedule: 500 Server Error: Internal Server Error for url: https://uat-rest.planbition.nl/api/ScheduleEmployeeShiftDemand?filter=contains%28EmployeeNumber%2C+%27RE0036443%27%29&PageNumber=1&PageSize=500


---Tool execution completed. Result length: 2---
---Start Answer Agent---


In [12]:
result["retrieved_data"]

'[]'

In [13]:
# Comprehensive evaluation of the agent system
print("=== COMPLETE SYSTEM EVALUATION ===")
print(f"Final result type: {type(result)}")
print(f"Result keys: {list(result.keys())}")
print(f"\nData retrieved successfully: {'Yes' if result.get('retrieved_data') else 'No'}")
print(f"Retrieved data length: {len(str(result.get('retrieved_data', '')))}")
print(f"Error occurred: {'Yes' if result.get('error_message') else 'No'}")

# Show sample of retrieved data
if result.get('retrieved_data'):
    preview = str(result['retrieved_data'])[:200]
    print(f"\nData preview: {preview}...")

print("\n" + "="*50)
result

=== COMPLETE SYSTEM EVALUATION ===
Final result type: <class 'dict'>
Result keys: ['date', 'messages', 'candidate_id', 'employee_number', 'next_action', 'retrieved_data', 'error_message', 'attempts', 'max_retries']

Data retrieved successfully: Yes
Retrieved data length: 2
Error occurred: No

Data preview: []...



{'date': '',
 'messages': [HumanMessage(content="what's my schedule for next week?", additional_kwargs={}, response_metadata={})],
 'candidate_id': '100102195',
 'employee_number': 'RE0036443',
 'next_action': 'ActionExecutionAgent',
 'retrieved_data': '[]',
 'error_message': None,
 'attempts': 0,
 'max_retries': 3}

## System Performance Analysis

### ✅ Working Components:
- **Module Reloading**: Successfully imports latest code changes
- **LLM Integration**: Azure OpenAI models responding correctly
- **Intent Classification**: Routes payslip requests to ActionExecutionAgent
- **Tool Binding**: LLM correctly selects get_payslip tool
- **API Integration**: Kentro client retrieving real payslip data
- **Graph Execution**: Complete supervisor → specialist → answer flow

### ⚠️ Issues Identified:
- **Double Execution**: Tool appears to run twice (investigate ToolNode behavior)
- **Debug Output**: Excessive logging needs cleanup for production
- **Error Handling**: Need more graceful failure modes

### 📊 Test Coverage:
- [x] ActionExecutionAgent direct testing
- [x] ChatSupervisor full workflow
- [ ] InformationRetrievalAgent testing
- [ ] FallbackTool testing
- [ ] Error scenarios testing

In [ ]:
# Test intent classification for different message types
print("=== INTENT CLASSIFICATION TESTING ===")

test_scenarios = [
    ("What's my schedule next week?", "ActionExecutionAgent"),
    ("How do I request vacation?", "InformationRetrievalAgent"), 
    ("Show me my contracts", "ActionExecutionAgent"),
    ("What's the weather today?", "FallbackTool"),
    ("Company policy on sick leave", "InformationRetrievalAgent"),
    ("Get my reservations", "ActionExecutionAgent")
]

for message, expected_route in test_scenarios:
    test_state_classification = {
        "date": datetime.now().strftime("%Y-%m-%d"),
        "messages": [HumanMessage(content=message)],
        "candidate_id": "test_id",
        "employee_number": "test_emp", 
        "next_action": "",
        "retrieved_data": "",
        "error_message": "",
        "attempts": 0,
        "max_retries": 3
    }
    
    result_classification = agent.classify_intent_node(test_state_classification)
    actual_route = result_classification.get('next_action')
    
    status = "✅" if actual_route == expected_route else "❌"
    print(f"{status} '{message[:30]}...' → {actual_route} (expected: {expected_route})")

In [ ]:
# Performance and timing analysis
import time

print("=== PERFORMANCE ANALYSIS ===")

# Time the full workflow
start_time = time.time()

performance_test_state = {
    "date": datetime.now().strftime("%Y-%m-%d"),
    "messages": [HumanMessage(content="What's my last payslip?")],
    "candidate_id": os.environ.get("KENTRO_TEST_CANDIDATE_ID"),
    "employee_number": os.environ.get("PLANBITION_TEST_EMPLOYEE_NUM"),
    "next_action": "",
    "retrieved_data": "",
    "error_message": "",
    "attempts": 0,
    "max_retries": 3
}

performance_result = agent.graph.invoke(performance_test_state)
end_time = time.time()

execution_time = end_time - start_time
print(f"Total execution time: {execution_time:.2f} seconds")
print(f"Data retrieval successful: {'Yes' if performance_result.get('retrieved_data') else 'No'}")
print(f"Result data size: {len(str(performance_result.get('retrieved_data', '')))} characters")

# Analyze state transitions
print(f"\nFinal state contains:")
for key, value in performance_result.items():
    if key == 'retrieved_data':
        print(f"  {key}: {len(str(value))} characters of data")
    elif key == 'messages':
        print(f"  {key}: {len(value)} messages")
    else:
        print(f"  {key}: {value}")